In [ ]:
from pyjstat import pyjstat
import requests
import pandas as pd
import matplotlib.pyplot as plt

ICT_SIZE_URL = 'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/tin00074?nace_r2=ICT'
CLOUD_SERVICES_URL = 'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/isoc_cicce_use?sizen_r2=M_C10_S951_XK&sizen_r2=L_C10_S951_XK&unit=PC_ENT&indic_is=E_CC'
GDP_DATA_FILE = 'gdp_data.csv'
FILLER = 1

# Function to display dataframes nicely and entirely
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    display(x)
    pd.reset_option('display.max_rows')

In [ ]:
# Pull json data about the size of ICT sector in the country and convert it to dataframe
ict = pyjstat.Dataset.read(ICT_SIZE_URL)
ict_df = ict.write('dataframe')

# Pull json data about the amount of cloud services used in country’s enterprises and convert it to dataframe
cloud = pyjstat.Dataset.read(CLOUD_SERVICES_URL)
cloud_df = cloud.write('dataframe')

# Drop unnecessary columns and fill missing values
ict_values = ict_df[["geo","time","value"]]
ict_values = ict_values.fillna(FILLER)

# We also need to aggregate (sum) values for cloud data since we want the overall values
cloud_values = cloud_df[["geo","time","value"]]
sum_cloud_values = cloud_values.groupby(['geo','time']).sum()
sum_cloud_values = sum_cloud_values.fillna(FILLER)

# Merge the datasets (join based on year and country)
ict_cloud = pd.merge(ict_values, sum_cloud_values, left_on=['geo','time'], right_on = ['geo','time'])
ict_cloud = ict_cloud.rename(columns={"value_x": "ict", "value_y": "cloud"})

# Replace zeros with ones since we want to retain some information
ict_cloud = ict_cloud.replace(0, 1)

print_full(ict_cloud)

In [ ]:
# Import gdp data from provided CSV file
gdp_data_df = pd.read_csv (GDP_DATA_FILE,delimiter='|')

# Remove lines for countries that already have offices
gdp_data_df = gdp_data_df[~gdp_data_df['2008'].str.contains("Office", na=False)]

# Pivot dataframe to make the merge possible, rename columns and sort data
gdp_pivot_df = gdp_data_df.melt(['Country'], var_name='time')
gdp_pivot_df = gdp_pivot_df.rename(columns={"value": "gdp", "Country": "geo"})
gdp_pivot_df = gdp_pivot_df.sort_values(by=['geo', 'time'])

# To use gdp data we need to replace commas with dots and cast as float. We also fill missing values with ones
gdp_pivot_df['gdp'] = gdp_pivot_df['gdp'].str.replace(',','.')
gdp_pivot_df['gdp'] = gdp_pivot_df['gdp'].astype(float)
gdp_pivot_df = gdp_pivot_df.fillna(1)

print_full(gdp_pivot_df)

In [ ]:
# Merge the two above dataframes (ict&cloud data, gdp data)
ict_cloud_gdp = pd.merge(ict_cloud, gdp_pivot_df, left_on=['geo','time'], right_on = ['geo','time'])

# Calculate attractiveness based on the supplied formula (GDP * percentage of ICT sector from GDP * usage of cloud computing in enterprises in a country)
ict_cloud_gdp['attract'] = ict_cloud_gdp['ict'] * ict_cloud_gdp['cloud'] * ict_cloud_gdp['gdp']

print_full(ict_cloud_gdp)

In [ ]:
# Drop other columns
df_attract = ict_cloud_gdp[["geo","time","attract"]]

# Sort by attractiveness and year, desc 
df_attract = df_attract.sort_values(by=['time','attract'], ascending=False)

# Finally, print a subset of the results
year = df_attract["time"]
max_year = year.max()
df_attract_selection = df_attract[df_attract['time'].str.contains(max_year, na=False)]
print_full(df_attract_selection)

# Pivot and plot a subset of the results
df_selection = df_attract.iloc[:3]
df_selection = df_selection[["geo"]]
df_attract = pd.merge(df_attract, df_selection, left_on=['geo'], right_on = ['geo'])
df_attract = df_attract[~df_attract['time'].str.contains("2016", na=False)]
df_attract = df_attract.pivot_table('attract', 'time', 'geo')
df_attract.plot(figsize=(20,20))